In [ ]:
# Dependencies
import tweepy 
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
from datetime import datetime

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:

# Twitter API Credentials
consumer_key = "nvw0aFUEiWC4uAxzXylujbai8"
consumer_secret = "9nooHblTgmaDU64wGxLs2bfw9lG6zj8qZvxzeHDGvLIZoUB2mB"
access_token = "40598976-5WYNZ8hYe0Le9gqFfKyScapq9v1MsxtY9JbEjlsxp"
access_token_secret = "JRWbqPLCjkvNd55NSTRR8NaHTdCIayQauUO5pAIMaQ3EU"

#Authenticate
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [ ]:
# Setup Vars
# get the favorite Users' Tweets
tUserList  = ["BBCNews","CNN","FoxNews","nytimes","CBS"]
# tUser = "ashish nachane"

# Real Person Filter 
minTweets = 5
maxTweets = 10000
maxFollowing = 2500
maxFollowers = 2500
lang = 'en'

# tUser = "ashish nachane"
userLst=[]
compLst=[]
posLst=[]
neuLst =[]
negLst=[]
timeLst=[]
allNews=[]

# declare the mean variables
posMean = 0.0
neuMean = 0.0
negMean = 0.0
compMean = 0.0
newsDf = pd.DataFrame()
allNewsLinesDf = pd.DataFrame()

In [ ]:
# Main Block

# Get all tweets from home feed
for tUser in tUserList:
    userLst=[]
    compLst=[]
    posLst=[]
    neuLst =[]
    negLst=[]
    timeLst=[]
    # public_tweets = api.user_timeline(tUer)
    print("Processing for: ", tUser)

    #llop through ATLEAST 100 tweets maybe 200 is better?
    for x in range(5):

        my_tweets = api.user_timeline(tUser)

        for tweet in my_tweets:
            # print(json.dumps(tweet, sort_keys=True, indent=4, separators=(',', ': ')))
            txt = tweet['text']
            # get the tweet Sentiment
            try:
                # print(txt)
                # compound = analyzer.polarity_scores(txt)
                # print(compound)
                tweetTime = datetime.strptime(tweet["created_at"], "%a %b %d %H:%M:%S %z %Y")
                compound = analyzer.polarity_scores(txt)["compound"]
                pos = analyzer.polarity_scores(txt)["pos"]
                neu = analyzer.polarity_scores(txt)["neu"]
                neg = analyzer.polarity_scores(txt)["neg"]
                timeLst.append(tweetTime)
                userLst.append(tUser)
                compLst.append(compound)
                posLst.append(pos)
                neuLst.append(neu)
                negLst.append(neg)
                # create list to hold all tweets for the news channel
                print("Adding to allNews for:" + tUser)
                allNews.append((tUser, txt, tweetTime, 0 , compound, pos,neu,neg))
            except:
                print("Dang it something went wrong: ", tUser)
           
    #Create dataframe to hold the details 
    allNewsLinesDf = pd.DataFrame(allNews, columns=["News Site", "Tweet", "Time","Past Tweets", 
                                                    "Compound","Positive","Neutral","Negative"])

    # create dataframe to hold all summary
    airSenti = {"News Site": tUser,  "Compound": np.mean(compLst), "Positives": np.mean(posLst), 
                "Negatives": np.mean(negLst),"Neutrsls": np.mean(neuLst), 
                "Score": ((np.mean(compLst)+np.mean(posLst)+np.mean(negLst)+np.mean(neuLst))/4)
                }
    newsDf = newsDf.append(airSenti, ignore_index=True)

In [ ]:
allNewsLinesDf.head()

In [ ]:
allNewsLinesDf = allNewsLinesDf.sort_values(["News Site","Time"],ascending =False)
# allNewsLinesDf["Past Tweets"] = allNewsLinesDf.groupby(["News Site","Time"]).rank(ascending=True)

aSite=""
for index,row in allNewsLinesDf.iterrows():
#     print ("Data IS: " + row["News Site"])
#     print ("Starting with: " + aSite)
#     print(row["News Site"] ==aSite)
#     aSite = row["News Site"]
    if (row["News Site"] != aSite):
        ctr = 1
        aSite = row["News Site"]
        allNewsLinesDf.set_value(index, "Past Tweets",ctr)
#         ctr += 1
    else:
        ctr += 1
        allNewsLinesDf.set_value(index, "Past Tweets",ctr)
        
# allNewsLinesDf

In [ ]:
# Plot the Moods bar
x_axis = allNewsLinesDf.loc[allNewsLinesDf["News Site"] == "nytimes"]["Past Tweets"]
y_axis = allNewsLinesDf.loc[allNewsLinesDf["News Site"] == "nytimes"]["Compound"]
plot.scatter(x_axis, y_axis, color='gold', alpha=0.80, label="NYT")
x_axis = allNewsLinesDf.loc[allNewsLinesDf["News Site"] == "BBCNews"]["Past Tweets"]
y_axis = allNewsLinesDf.loc[allNewsLinesDf["News Site"] == "BBCNews"]["Compound"]
plot.scatter(x_axis, y_axis, color='c',alpha=0.80,label="BBC")
x_axis = allNewsLinesDf.loc[allNewsLinesDf["News Site"] == "CNN"]["Past Tweets"]
y_axis = allNewsLinesDf.loc[allNewsLinesDf["News Site"] == "CNN"]["Compound"]
plot.scatter(x_axis, y_axis, color='g', alpha=0.80,label="CNN")
x_axis = allNewsLinesDf.loc[allNewsLinesDf["News Site"] == "FoxNews"]["Past Tweets"]
y_axis = allNewsLinesDf.loc[allNewsLinesDf["News Site"] == "FoxNews"]["Compound"]
plot.scatter(x_axis, y_axis, color='r',alpha=0.80, label="FOX")
x_axis = allNewsLinesDf.loc[allNewsLinesDf["News Site"] == "CBS"]["Past Tweets"]
y_axis = allNewsLinesDf.loc[allNewsLinesDf["News Site"] == "CBS"]["Compound"]
plot.scatter(x_axis, y_axis, color='black',alpha=0.80, label="CBS")
plot.xlabel("Tweets")
plot.ylabel("Polarity")
plot.legend(loc="best")
plot.show()

In [ ]:
# Scatter Plot the 
x_axis = np.arange(len(newsDf))
y_axis = newsDf["Compound"]
colors = ['c','g','r','gold','black']
plot.bar(x_axis, y_axis, color=colors, alpha=0.5, align="edge")
# Tell matplotlib where we would like to place each of our x axis headers
tick_locations = [value+0.4 for value in x_axis]
plot.xticks(tick_locations, ["BBC", "CNN", "Fox", "NYT", "CBS"])
# Give our chart some labels and a tile
plot.title("Overal Media Sentiment on Twitter")
plot.xlabel("Media Outlets ")
plot.ylabel("Polarity")
plot.show()